In [ ]:
import overpy
import geopandas as gpd
from shapely.geometry import Polygon, box
from tqdm import tqdm

In [ ]:
def way_to_polygon(way):
    try:
        coords = [(node.lon, node.lat) for node in way.nodes]
        
        if coords:
            polygon = Polygon(coords)
            land_use = way.tags.get("landuse")
        else:
            return None, "Failed"
    except:
        return None, "Exception"
    return polygon, land_use

bbox = box(55.66731601212282, 12.560585920836248, 55.699960803118294, 12.607727632258538)
api = overpy.Overpass()

query = f"""
[out:json];
wr["landuse"]({bbox.bounds[0]}, {bbox.bounds[1]}, {bbox.bounds[2]}, {bbox.bounds[3]});
(._;>;);
out body;
"""

result = api.query(query)

geoms = []
land_uses = []
for way in tqdm(result.ways, bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}"):
    polygon, land_use = way_to_polygon(way)
    geoms.append(polygon)
    land_uses.append(land_use)

gdf = gpd.GeoDataFrame({"geometry": geoms, "landuse": land_uses}, crs="EPSG:4326")

In [ ]:
import osmnx as ox

gdf = ox.features_from_bbox(bbox.bounds, tags={"landuse": True})